In [2]:
import cv2
import numpy as np
import os

# showing image
def show_image(image, title="Image", scale=5):
    h, w = image.shape
    resized_image = cv2.resize(image, (w * scale, h * scale), interpolation=cv2.INTER_NEAREST)
    cv2.imshow(title, resized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# load data

final_data = "C:/Users/HUY/Desktop/my_project/final_data"
synthetic_path = "C:/Users/HUY/Desktop/my_project/synthetic_data"

# tải dữ liệu dựa theo góc của dáng đi
def load_view(folder_paths, view):
    train, test = [], []
    train_view = []
    test_view = []
    for folder in os.listdir(folder_paths):
        if folder == "train":
            tr_labels = []
            for img in os.listdir(f"{folder_paths}/{folder}"):
                if view in img:
                    im = np.array(cv2.imread(f"{folder_paths}/{folder}/{img}", cv2.IMREAD_GRAYSCALE))
                    tr_labels.append(img[0:3])
                    train.append(im)
                    train_view.append(view)
        if folder == "test":
            te_labels = []
            for img in os.listdir(f"{folder_paths}/{folder}"):
                if view in img:
                    im = np.array(cv2.imread(f"{folder_paths}/{folder}/{img}", cv2.IMREAD_GRAYSCALE))
                    te_labels.append(img[0:3])
                    test.append(im)
                    test_view.append(view)
    print(f"Hoàn thành tải dữ liệu lên từ {folder_paths} với góc {view} độ")
    return train, test, tr_labels, te_labels, train_view, test_view


# tải dữ liệu dáng đi với tất cả các góc
def load_data(view = "crossview"):
    
    views = ["000","018","036","054","072","090","108","126","144","162","180"]
    x_train, y_train = [], []
    x_test, y_test = [], []
    train_view, test_view = [], []
    
    if view in views:
        views = [view]
    
    for view in views:
        x_rtrain_view, x_rtest_view, y_rtrain_view, y_rtest_view, rtrain_view, rtest_view = load_view(final_data, view)
        x_strain_view, x_stest_view, y_strain_view, y_stest_view, strain_view, stest_view = load_view(synthetic_path, view)
        
        x_train = x_train + x_rtrain_view + x_strain_view
        y_train = y_train + y_rtrain_view + y_strain_view
        x_test = x_test + x_rtest_view + x_stest_view
        y_test = y_test + y_rtest_view + y_stest_view
        train_view = train_view + rtrain_view + strain_view
        test_view = test_view + rtest_view + stest_view
        
    print(f"✅ Kích thước tập train và nhãn: {len(x_train)} và {len(y_train)}")
    print(f"✅ Kích thước tập test và nhãn: {len(x_test)} và {len(y_test)}")
    return x_train, y_train, x_test, y_test, train_view, test_view

In [3]:
x_train, y_train, x_test, y_test, train_view, test_view = load_data(view = "crossview")

Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/final_data với góc 000 độ
Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/synthetic_data với góc 000 độ
Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/final_data với góc 018 độ
Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/synthetic_data với góc 018 độ
Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/final_data với góc 036 độ
Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/synthetic_data với góc 036 độ
Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/final_data với góc 054 độ
Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/synthetic_data với góc 054 độ
Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/final_data với góc 072 độ
Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/synthetic_data với góc 072 độ
Hoàn thành tải dữ liệu lên từ C:/Users/HUY/Desktop/my_project/final_data với góc 090 độ
Hoàn thành t

In [113]:
import pandas as pd
from sklearn.decomposition import PCA  

# 95 -> 14 ~ 13
# 96 -> 15 ~ 14
# 97 -> 16 ~ 15
# 98 -> 17 ~ 16
# 99 -> 18 ~ 17
# ==> chọn số component = 16 = 2^4

pca = PCA(n_components = 16)
x_train = pca.fit_transform([x.flatten() for x in x_train])
x_test = pca.transform([x.flatten() for x in x_test])

print(f"Kích thước của tập train và test sau khi PCA: {x_train.shape} {x_test.shape}")

data_path = ['data_train.csv', 'data_test.csv']

df_train = pd.DataFrame(x_train)
df_train['label'] = y_train
df_train['view'] = train_view
df_train.to_csv( data_path[0], index=False)

df_test = pd.DataFrame(x_test)
df_test['label'] = y_test
df_test['view'] = test_view
df_test.to_csv( data_path[1], index=False)

for path in data_path:
    if os.path.exists(path):
        print(f"✅ Đã tạo file {path} thành công!")

Kích thước của tập train và test sau khi PCA: (5668, 16) (2834, 16)
✅ Đã tạo file data_train.csv thành công!
✅ Đã tạo file data_test.csv thành công!


In [ ]:
# Đã hoàn thành trích xuất đặc trưng bằng PCA cho dataset
# Lưu các đặc trưng đó thành các file csv để xử lý sau
# ===> OKE